<a href="https://colab.research.google.com/github/ShuqairABD/GRADIO_Yolov8_seg_with_Dicom_jpg_png/blob/main/Abdalla_Shuqair/GRADIO_Yolov8_seg_with_Dicom_jpg_png_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Start**

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
!pip install pillow

In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import PIL.Image as Image
from ultralytics import ASSETS, YOLO
import os
import numpy as np
import pydicom
import torch
from PIL import Image
from ultralytics import YOLO
from io import BytesIO
from pathlib import Path


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## **Gradio**

In [ ]:

#  YOLO weights
model = YOLO('/content/drive/MyDrive/yolov8-seg_2_best.pt')

def predict_image(img, conf_threshold, iou_threshold):
    # DICOM в PIL
    if isinstance(img, str) and img.lower().endswith('.dcm'):
        dicom_image = pydicom.dcmread(img)
        image_array = dicom_image.pixel_array

        if image_array.max() > 1:
            image_array = (image_array / image_array.max() * 255).astype(np.uint8)
        img_pil = Image.fromarray(image_array).convert('RGB')
    else:
        img_pil = Image.open(img).convert('RGB')


    results = model.predict(
        source=img_pil,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    # result
    for r in results:
        im_array = r.plot()
        processed_image = Image.fromarray(im_array[..., ::-1])

    return img_pil, processed_image

#Gradio
iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.File(label="Загрузить изображение (DCM, JPG, PNG, JPEG, webp, tiff...)"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="conf_threshold - Порог доверия для обнаружения объектов."),
        gr.Slider(minimum=0, maximum=1, value=0.45, label="iou_threshold - Порог пересечения для разделения объектов."),
    ],
    outputs=[
        gr.Image(type="pil", label="Исходное изображение"),
        gr.Image(type="pil", label="Обработанное изображение"),
    ],
    title="Автоматическая сегментация с использованием YOLOv8",
    description="Загрузите изображение (формат DCM, JPG или PNG) для анализа."

)

if __name__ == "__main__":
    iface.launch()


*** with ASSETS**

In [ ]:

# YOLO
model = YOLO('/content/drive/MyDrive/yolov8-seg_2_best.pt')

ASSETS = "/content/drive/MyDrive"

def predict_image(img, conf_threshold, iou_threshold):
    # DICOM в PIL
    if isinstance(img, str) and img.lower().endswith('.dcm'):
        dicom_image = pydicom.dcmread(img)
        image_array = dicom_image.pixel_array

        if image_array.max() > 1:
            image_array = (image_array / image_array.max() * 255).astype(np.uint8)
        img_pil = Image.fromarray(image_array).convert('RGB')
    else:
        img_pil = Image.open(img).convert('RGB')

    results = model.predict(
        source=img_pil,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    for r in results:
        im_array = r.plot()
        processed_image = Image.fromarray(im_array[..., ::-1])

    return img_pil, processed_image

# Gradio
iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.File(label="Загрузить изображение (DCM, JPG, PNG, JPEG, webp, tiff...)"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="conf_threshold - Порог доверия для обнаружения объектов."),
        gr.Slider(minimum=0, maximum=1, value=0.45, label="iou_threshold - Порог пересечения для разделения объектов."),
    ],
    outputs=[
        gr.Image(type="pil", label="Исходное изображение"),
        gr.Image(type="pil", label="Обработанное изображение"),
    ],
    title="Автоматическая сегментация с использованием YOLOv8-seg",
    description="Загрузить изображение (формат DCM, JPG, PNG, JPEG, webp, tiff..) для анализа или попробовать EXAMPLES ниже...",
    examples=[
        [str(Path(ASSETS) / "16.jpg"), 0.25, 0.45],
        [str(Path(ASSETS) / "61.jpg"), 0.25, 0.45],
        [str(Path(ASSETS) / "711.jpg"), 0.25, 0.45],
    ],
)

if __name__ == "__main__":
    iface.launch()


## **Gradio adding CSS / dicom / jpg / png...**

In [ ]:

# YOLO
model = YOLO('/content/drive/MyDrive/yolov8-seg_2_best.pt')

ASSETS = "/content/drive/MyDrive"


def predict_image(img, conf_threshold, iou_threshold):

    # DICOM ------> PIL
    if isinstance(img, str) and img.lower().endswith('.dcm'):
        dicom_image = pydicom.dcmread(img)
        image_array = dicom_image.pixel_array

        if image_array.max() > 1:
            image_array = (image_array / image_array.max() * 255).astype(np.uint8)
        img_pil = Image.fromarray(image_array).convert('RGB')
    else:
        img_pil = Image.open(img).convert('RGB')

    results = model.predict(
        source=img_pil,
        conf=conf_threshold,
        iou=iou_threshold,
        show_labels=True,
        show_conf=True,
        imgsz=640,
    )

    for r in results:
        im_array = r.plot()
        processed_image = Image.fromarray(im_array[..., ::-1])

    return img_pil, processed_image

# CSS
custom_css = """
@keyframes fadeIn {
  0% { opacity: 0; }
  100% { opacity: 1; }
}

h1 {
  animation: fadeIn 3.5s ease-in-out;
  font-size: 2.5em;
  color: #3246a1;
  text-align: center;
}
"""

# Gradio
iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.File(label="Загрузить изображение (DCM, JPG, PNG, JPEG, webp, tiff...)"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="conf_threshold - Порог доверия для обнаружения объектов."),
        gr.Slider(minimum=0, maximum=1, value=0.45, label="iou_threshold - Порог пересечения для разделения объектов."),
    ],
    outputs=[
        gr.Image(type="pil", label="Исходное изображение"),
        gr.Image(type="pil", label="Обработанное изображение"),
    ],

    title="Автоматическая сегментация с использованием YOLOv8-seg",
    description="Загрузить изображение (формат DCM, JPG, PNG, JPEG, webp, tiff..) для анализа или попробовать EXAMPLES ниже...",
    examples=[
        [str(Path(ASSETS) / "16.jpg"), 0.25, 0.45],
        [str(Path(ASSETS) / "61.jpg"), 0.25, 0.45],
        [str(Path(ASSETS) / "185.dcm"), 0.25, 0.45],
        [str(Path(ASSETS) / "999.png"), 0.25, 0.45],
        [str(Path(ASSETS) / "conv.tiff"), 0.25, 0.45],
        [str(Path(ASSETS) / "711.jpg"), 0.25, 0.45],
    ],
    css=custom_css,  # CSS
)

if __name__ == "__main__":
    iface.launch()
